In [1]:
import keras
from keras.models import Sequential
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import  BatchNormalization
from keras.layers.convolutional import *
from keras import regularizers
import numpy as np
import tensorflow as tf
import  matplotlib.pyplot as plt
import matplotlib
import itertools
from numpy import*
from sklearn.utils import shuffle
%matplotlib inline
import random as rn
import os
os.environ['PYTHONASHSEED']= '0'

Using TensorFlow backend.


In [2]:

#################################################
def CNN_model(input_tensor=None, input_shape=None, pooling=None, classes=5):
    
    # Determine proper input shape
    if input_shape == None:
        input_shape = (200,200,3)
    
    img_input = Input(input_shape)

    # Block 1
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block11_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block11_pool')(x)

    # Block 2
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block12_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block12_pool')(x)
    x = Dropout(0.5)(x) 
    # Block 3
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block13_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block13_pool')(x)
    x = Dropout(0.5)(x) 
    
    # Classification block
    x = Flatten(name='flatten1')(x)
    x = Dense(200, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='fc11')(x)
    x = Dropout(0.5)(x) 
    x = Dense(200, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='fc12')(x)
    x = Dropout(0.5)(x) 
    x = Dense(classes, activation='softmax', name='predictions1')(x)


    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='hand_model')
    return model

In [3]:
posture_model = CNN_model()

In [4]:
posture_model.load_weights("handposture1.weights.best.hdf5")
# posture_model.load_weights("hand_model_weights.h5")

In [5]:
import matplotlib.pyplot as plt

In [7]:
classes = ['A','B','C','Point','V']


In [8]:
valid_path = 'Data/Test'
test_datagen=ImageDataGenerator(rescale=1./255)
valid_batches = test_datagen.flow_from_directory(valid_path, target_size = (200,200), classes = classes, batch_size = 1, shuffle=False)

Found 259 images belonging to 5 classes.


In [9]:
result = posture_model.predict_generator(valid_batches)


In [ ]:
pred = np.argmax(result, axis=1)

In [ ]:
pred

In [ ]:
valid_batches.classes


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
accuracy_score(valid_batches.classes, pred)

In [ ]:
valid_batches.filenames[-1]

In [ ]:
confusion_matrix(valid_batches.classes, pred)

In [ ]:
classes = ['A','B','C','Point','V']
import cv2
from PIL import Image

frame_path = '.\Data\\Test\\Point\\2.jpg'
im = Image.open(frame_path) 
im = im.resize((200,200))
plt.imshow(im)
imarr = np.array(im).astype(np.float32)
imarr = np.expand_dims(imarr/255, axis=0)
print(imarr.shape)
proba = posture_model.predict(imarr, verbose=0)
pred_cls = np.argmax(proba,axis=-1)  
print(proba)
print(pred_cls)
